In [1]:
import tensorflow as tf
from tensorflow.keras import layers, utils
from tensorflow import keras
from matplotlib import pyplot as plt

In [2]:
IMAGE_SIZE = 100
BATCH_SIZE = 32

In [3]:
train = utils.image_dataset_from_directory("datasets/data/split/VTID2/train", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
val = utils.image_dataset_from_directory("datasets/data/split/VTID2/val", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
test = utils.image_dataset_from_directory("datasets/data/split/VTID2/test", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)

Found 3048 files belonging to 5 classes.
Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Found 652 files belonging to 5 classes.
Found 656 files belonging to 5 classes.


2022-08-02 10:21:47.124863: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-02 10:21:47.124970: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
inputs = keras.Input(shape=(100, 100, 3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(32, 3, activation='relu')(inputs)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(128, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(256, 3, activation='relu')(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(5, activation='softmax')(x)

model = keras.Model(inputs, outputs)

model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 98, 98, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 23, 23, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 21, 21, 128)       73856 

In [5]:
callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=8
    ),
    keras.callbacks.ModelCheckpoint(
        filepath="models/vtid_1.h5",
        monitor="val_accuracy",
        save_best_only=True
    )
]

In [6]:
history = model.fit(
  train.cache(),
  epochs=40,
  callbacks=callbacks_list,
  validation_data=val.cache()
)

Epoch 1/40


2022-08-02 10:21:47.827577: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-08-02 10:21:47.827679: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


96/96 [==============================] - ETA: 0s - loss: 2.7021 - accuracy: 0.3274

2022-08-02 10:21:52.738272: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


96/96 [==============================] - 6s 53ms/step - loss: 2.7021 - accuracy: 0.3274 - val_loss: 1.6024 - val_accuracy: 0.2623
Epoch 2/40
96/96 [==============================] - 8s 85ms/step - loss: 1.3381 - accuracy: 0.4836 - val_loss: 1.0424 - val_accuracy: 0.5951
Epoch 3/40
96/96 [==============================] - 4s 45ms/step - loss: 0.9600 - accuracy: 0.6437 - val_loss: 0.8715 - val_accuracy: 0.6564
Epoch 4/40
96/96 [==============================] - 4s 43ms/step - loss: 0.7089 - accuracy: 0.7618 - val_loss: 0.4945 - val_accuracy: 0.8206
Epoch 5/40
96/96 [==============================] - 4s 43ms/step - loss: 0.4604 - accuracy: 0.8406 - val_loss: 0.3930 - val_accuracy: 0.8512
Epoch 6/40
96/96 [==============================] - 4s 45ms/step - loss: 0.3472 - accuracy: 0.8829 - val_loss: 0.2220 - val_accuracy: 0.9233
Epoch 7/40
96/96 [==============================] - 4s 43ms/step - loss: 0.2563 - accuracy: 0.9255 - val_loss: 1.1773 - val_accuracy: 0.6058
Epoch 8/40
96/96 [======

In [7]:
model.evaluate(test)

21/21 [==============================] - 1s 21ms/step - loss: 0.0401 - accuracy: 0.9893


[0.040077876299619675, 0.9893292784690857]

In [8]:
load_model = keras.models.load_model("models/vtid_1.h5")
load_model.evaluate(test)

 1/21 [>.............................] - ETA: 4s - loss: 0.0068 - accuracy: 1.0000

2022-08-02 10:23:34.012779: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


21/21 [==============================] - 1s 20ms/step - loss: 0.0596 - accuracy: 0.9909


[0.05964948609471321, 0.9908537268638611]